In [1]:
from layoutlm import FunsdDataset, LayoutlmConfig, LayoutlmForTokenClassification
from transformers import BertTokenizer, AutoTokenizer
from layoutlm.data.funsd import read_examples_from_file, convert_examples_to_features

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from tqdm import tqdm, trange
import numpy as np

In [3]:
def get_labels(path):
    with open(path, "r") as f:
        labels = f.read().splitlines()
    if "O" not in labels:
        labels = ["O"] + labels
    return labels

# load data

In [2]:
tokenizer = AutoTokenizer.from_pretrained('model/layoutlm-base-uncased/')

In [4]:
labels = get_labels('data/infer_data/labels.txt')
num_labels = len(labels)
pad_token_label_id = CrossEntropyLoss().ignore_index

In [5]:
examples = read_examples_from_file('data/infer_data/', 'test')
features = convert_examples_to_features(
    examples, labels, 512,
    tokenizer, cls_token_at_end=False, # xlnet has a cls token at the end
    cls_token=tokenizer.cls_token, cls_token_segment_id=0,
    sep_token=tokenizer.sep_token, sep_token_extra=False,
    # roberta uses an extra separator b/w pairs of sentences, cf. github.com/pytorch/fairseq/commit/1684e166e3da03f5b600dbb7855cb98ddfcd0805
    pad_on_left=False,
    # pad on the left for xlnet
    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
    pad_token_segment_id=0,
    pad_token_label_id=pad_token_label_id,
)

In [6]:
all_input_ids = torch.tensor(
    [f.input_ids for f in features], dtype=torch.long
)
all_input_mask = torch.tensor(
    [f.input_mask for f in features], dtype=torch.long
)
all_segment_ids = torch.tensor(
    [f.segment_ids for f in features], dtype=torch.long
)
all_label_ids = torch.tensor(
    [f.label_ids for f in features], dtype=torch.long
)
all_bboxes = torch.tensor([f.boxes for f in features], dtype=torch.long)

# Prepare model

In [7]:
model = LayoutlmForTokenClassification.from_pretrained('model/layoutlm-base-uncased/')
model.to('cuda')
model.eval();

Some weights of the model checkpoint at model/layoutlm-base-uncased/ were not used when initializing LayoutlmForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing LayoutlmForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutlmForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LayoutlmForTokenClassification were not initialized from the model checkpoint at model/layoutlm-base-uncased/ and are newly i

In [16]:
class LayoutLM(nn.Module):
    def __init__(self, model):
        super(LayoutLM, self).__init__()
        self.model = model
        
    def forward(self, input_ids, bbox, attention_mask, token_type_ids):
        return self.model(
            input_ids, bbox, attention_mask, token_type_ids
        )[0]

In [17]:
layout_model = LayoutLM(model)
layout_model.to('cuda')
layout_model.eval();

In [18]:
inputs = {
    'input_ids': all_input_ids[0].unsqueeze(0).to('cuda'),
    'attention_mask': all_input_mask[0].unsqueeze(0).to('cuda'),
    'token_type_ids': all_segment_ids[0].unsqueeze(0).to('cuda'),
    'bbox': all_bboxes[0].unsqueeze(0).to('cuda')
}

In [19]:
with torch.no_grad():
    outputs = layout_model(**inputs)

# Convert to onnx model

In [23]:
sample_inputs = (
    inputs['input_ids'], inputs['bbox'], 
    inputs['attention_mask'], inputs['token_type_ids'], 
)
torch.onnx.export(
    layout_model, sample_inputs, 'deployment/layoutlm_onnx/1/model.onnx', export_params=True,
    opset_version=11, do_constant_folding=True,
    input_names=['input_ids', 'bbox', 'attention_mask', 'token_type_ids'], output_names=['output'],
    dynamic_axes={
        'input_ids' : {0 : 'batch_size'},    # variable lenght axes
        'bbox' : {0 : 'batch_size'},
        'attention_mask' : {0 : 'batch_size'},
        'token_type_ids' : {0 : 'batch_size'},
        'output' : {0 : 'batch_size'}
    }
)

/home/superceed1/anaconda3/envs/py38torch17/lib/python3.8/site-packages/transformers/modeling_utils.py:2158: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert all(
